# 20 - Trino: Advanced Analytics

Zaawansowana analityka w Trino - funkcje okienkowe, funkcje przybliżone, optymalizacja.

**Tematy:**
- Window functions w Trino (porównanie z notebook 05 Spark SQL)
- Approximate functions: approx_distinct, approx_percentile
- Tablice i mapy w Trino SQL
- Trino + dbt - transformacje SQL jako kod
- Materialized views
- Partycjonowanie i bucketing w Hive connector
- Optymalizacja: partition pruning, predicate pushdown
- Performance tuning: session properties, resource groups

## 1. Setup

In [ ]:
from trino.dbapi import connect
import pandas as pd
import time

conn = connect(
    host="trino",
    port=8080,
    user="analyst",
    catalog="postgresql",
    schema="movielens"
)

def run_query(sql, conn=conn):
    cursor = conn.cursor()
    cursor.execute(sql)
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    return pd.DataFrame(rows, columns=columns)

def run_query_print(sql, conn=conn, max_rows=20):
    df = run_query(sql, conn)
    print(f"Rows: {len(df)}")
    display(df.head(max_rows))
    return df

print("Connected to Trino!")

## 2. Window Functions w Trino

Window functions w Trino działają identycznie jak w Spark SQL (notebook 05). Składnia ANSI SQL.

Dostępne funkcje:
- `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()` - numeracja/ranking
- `LAG()`, `LEAD()` - poprzedni/następny wiersz
- `FIRST_VALUE()`, `LAST_VALUE()`, `NTH_VALUE()` - wartości z okna
- `SUM() OVER`, `AVG() OVER`, `COUNT() OVER` - agregaty okienkowe
- `NTILE(n)` - podział na grupy
- `CUME_DIST()`, `PERCENT_RANK()` - dystrybucja

In [ ]:
# ROW_NUMBER - top 3 filmy per użytkownik (identycznie jak w Spark SQL)
run_query_print("""
    SELECT user_id, movie_id, rating, rn
    FROM (
        SELECT user_id, movie_id, rating,
               ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY rating DESC) AS rn
        FROM ratings
    )
    WHERE rn <= 3 AND user_id <= 5
    ORDER BY user_id, rn
""")

In [ ]:
# RANK vs DENSE_RANK
run_query_print("""
    SELECT user_id, movie_id, rating,
           RANK() OVER (PARTITION BY user_id ORDER BY rating DESC) AS rank_val,
           DENSE_RANK() OVER (PARTITION BY user_id ORDER BY rating DESC) AS dense_rank_val
    FROM ratings
    WHERE user_id = 1
    LIMIT 15
""")

In [ ]:
# LAG / LEAD - zmiana oceny użytkownika w czasie
run_query_print("""
    SELECT user_id, movie_id, rating, rating_timestamp,
           LAG(rating) OVER (PARTITION BY user_id ORDER BY rating_timestamp) AS prev_rating,
           rating - LAG(rating) OVER (PARTITION BY user_id ORDER BY rating_timestamp) AS rating_diff
    FROM ratings
    WHERE user_id = 42
    ORDER BY rating_timestamp
    LIMIT 20
""")

In [ ]:
# Kumulatywna suma i średnia krocząca
run_query_print("""
    SELECT user_id, movie_id, rating, rating_timestamp,
           COUNT(*) OVER (
               PARTITION BY user_id ORDER BY rating_timestamp
           ) AS cumulative_count,
           ROUND(AVG(rating) OVER (
               PARTITION BY user_id
               ORDER BY rating_timestamp
               ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
           ), 2) AS moving_avg_5
    FROM ratings
    WHERE user_id = 42
    ORDER BY rating_timestamp
    LIMIT 20
""")

In [ ]:
# NTILE - podział na kwartyle
run_query_print("""
    WITH user_avg AS (
        SELECT user_id, AVG(rating) AS avg_rating, COUNT(*) AS cnt
        FROM ratings
        GROUP BY user_id
        HAVING COUNT(*) >= 50
    )
    SELECT quartile,
           COUNT(*) AS num_users,
           ROUND(MIN(avg_rating), 2) AS min_avg,
           ROUND(MAX(avg_rating), 2) AS max_avg
    FROM (
        SELECT *, NTILE(4) OVER (ORDER BY avg_rating) AS quartile
        FROM user_avg
    )
    GROUP BY quartile
    ORDER BY quartile
""")

In [ ]:
# CUME_DIST i PERCENT_RANK - funkcje dystrybucji (niedostępne łatwo w Spark SQL)
run_query_print("""
    WITH movie_stats AS (
        SELECT movie_id,
               COUNT(*) AS num_ratings,
               AVG(rating) AS avg_rating
        FROM ratings
        GROUP BY movie_id
        HAVING COUNT(*) >= 100
    )
    SELECT m.title,
           ms.num_ratings,
           ROUND(ms.avg_rating, 2) AS avg_rating,
           ROUND(CUME_DIST() OVER (ORDER BY ms.avg_rating), 3) AS cume_dist,
           ROUND(PERCENT_RANK() OVER (ORDER BY ms.avg_rating), 3) AS pct_rank
    FROM movie_stats ms
    JOIN movies m ON ms.movie_id = m.movie_id
    ORDER BY ms.avg_rating DESC
    LIMIT 15
""")

In [ ]:
# FIRST_VALUE / LAST_VALUE - unikalne dla Trino (łatwiejsze niż w Spark)
run_query_print("""
    SELECT user_id,
           movie_id,
           rating,
           rating_timestamp,
           FIRST_VALUE(rating) OVER (
               PARTITION BY user_id ORDER BY rating_timestamp
           ) AS first_rating,
           LAST_VALUE(rating) OVER (
               PARTITION BY user_id
               ORDER BY rating_timestamp
               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
           ) AS last_rating
    FROM ratings
    WHERE user_id <= 3
    ORDER BY user_id, rating_timestamp
    LIMIT 20
""")

## 3. Approximate Functions

Trino oferuje **funkcje przybliżone**, które działają znacznie szybciej na dużych zbiorach danych kosztem niewielkiej niedokładności.

| Funkcja | Dokładny odpowiednik | Algorytm | Błąd |
|---------|---------------------|----------|------|
| `approx_distinct()` | `COUNT(DISTINCT)` | HyperLogLog | ~2.3% |
| `approx_percentile(col, p)` | percentyl z sortowania | T-Digest / QDigest | konfigurowalny |
| `approx_most_frequent(n, col, cap)` | GROUP BY + ORDER BY + LIMIT | Count-Min Sketch | przybliżony |

Przydatne gdy:
- Dane są bardzo duże (miliardy wierszy)
- Potrzebna szybka odpowiedź (dashboard)
- Dokładność 98% wystarczy

In [ ]:
# approx_distinct vs COUNT(DISTINCT) - porównanie dokładności i szybkości
run_query_print("""
    SELECT
        COUNT(DISTINCT user_id) AS exact_users,
        approx_distinct(user_id) AS approx_users,
        COUNT(DISTINCT movie_id) AS exact_movies,
        approx_distinct(movie_id) AS approx_movies
    FROM ratings
""")

In [ ]:
# Benchmark: approx_distinct vs COUNT(DISTINCT)
for name, sql in [
    ("COUNT(DISTINCT)", "SELECT COUNT(DISTINCT user_id) FROM ratings"),
    ("approx_distinct", "SELECT approx_distinct(user_id) FROM ratings"),
]:
    start = time.time()
    run_query(sql)
    print(f"{name:<20} {time.time() - start:.3f}s")

In [ ]:
# approx_percentile - szybkie percentyle
run_query_print("""
    SELECT
        approx_percentile(rating, 0.25) AS p25,
        approx_percentile(rating, 0.50) AS median,
        approx_percentile(rating, 0.75) AS p75,
        approx_percentile(rating, 0.90) AS p90,
        approx_percentile(rating, 0.99) AS p99
    FROM ratings
""")

In [ ]:
# approx_percentile z wieloma wartościami naraz (tablica percentyli)
run_query_print("""
    SELECT movie_id,
           COUNT(*) AS cnt,
           ROUND(AVG(rating), 2) AS avg_rating,
           approx_percentile(rating, 0.5) AS median_rating,
           approx_percentile(rating, ARRAY[0.25, 0.5, 0.75]) AS quartiles
    FROM ratings
    GROUP BY movie_id
    HAVING COUNT(*) > 10000
    ORDER BY cnt DESC
    LIMIT 10
""")

In [ ]:
# approx_distinct per grupa - unikalni użytkownicy per ocena
run_query_print("""
    SELECT
        CAST(rating AS VARCHAR) AS rating_val,
        COUNT(*) AS total,
        approx_distinct(user_id) AS approx_unique_users,
        approx_distinct(movie_id) AS approx_unique_movies
    FROM ratings
    GROUP BY rating
    ORDER BY rating DESC
""")

## 4. Tablice i mapy w Trino SQL

Trino ma natywne wsparcie dla złożonych typów danych:
- `ARRAY` - lista wartości
- `MAP` - słownik klucz-wartość
- `ROW` - struktura (named tuple)

Przydatne do pracy z danymi semi-strukturalnymi (JSON, genres itp.).

In [ ]:
# SPLIT - rozdzielenie genres na tablicę
run_query_print("""
    SELECT title,
           genres,
           SPLIT(genres, '|') AS genre_array,
           CARDINALITY(SPLIT(genres, '|')) AS num_genres
    FROM movies
    LIMIT 10
""")

In [ ]:
# UNNEST - rozwinięcie tablicy do wierszy (odpowiednik explode w Spark)
run_query_print("""
    SELECT m.movie_id, m.title, genre
    FROM movies m
    CROSS JOIN UNNEST(SPLIT(m.genres, '|')) AS t(genre)
    WHERE m.movie_id <= 5
    ORDER BY m.movie_id, genre
""")

In [ ]:
# Analiza gatunków z UNNEST
run_query_print("""
    WITH genre_exploded AS (
        SELECT m.movie_id, m.title, genre
        FROM movies m
        CROSS JOIN UNNEST(SPLIT(m.genres, '|')) AS t(genre)
    )
    SELECT genre,
           COUNT(DISTINCT movie_id) AS num_movies
    FROM genre_exploded
    WHERE genre != '(no genres listed)'
    GROUP BY genre
    ORDER BY num_movies DESC
""")

In [ ]:
# ARRAY aggregation - zbierz oceny użytkownika do tablicy
run_query_print("""
    SELECT user_id,
           COUNT(*) AS num_ratings,
           ARRAY_AGG(rating ORDER BY rating_timestamp) AS rating_history,
           ARRAY_AGG(DISTINCT CAST(rating AS VARCHAR)) AS unique_ratings
    FROM ratings
    WHERE user_id <= 3
    GROUP BY user_id
""")

In [ ]:
# MAP - tworzenie map z danych
run_query_print("""
    SELECT user_id,
           MAP_AGG(movie_id, rating) AS movie_ratings_map
    FROM ratings
    WHERE user_id = 1
    GROUP BY user_id
""")

In [ ]:
# Operacje na tablicach: CONTAINS, ELEMENT_AT, ARRAY_JOIN
run_query_print("""
    SELECT title,
           genres,
           CONTAINS(SPLIT(genres, '|'), 'Comedy') AS is_comedy,
           CONTAINS(SPLIT(genres, '|'), 'Drama') AS is_drama,
           ELEMENT_AT(SPLIT(genres, '|'), 1) AS first_genre,
           ARRAY_JOIN(SPLIT(genres, '|'), ' / ') AS genres_formatted
    FROM movies
    LIMIT 10
""")

In [ ]:
# TRANSFORM - transformacja elementów tablicy (lambda w SQL!)
run_query_print("""
    SELECT title,
           SPLIT(genres, '|') AS genres_array,
           TRANSFORM(SPLIT(genres, '|'), x -> UPPER(x)) AS genres_upper,
           FILTER(SPLIT(genres, '|'), x -> LENGTH(x) > 5) AS long_genres
    FROM movies
    LIMIT 10
""")

## 5. Trino + dbt - transformacje SQL jako kod

### Czym jest dbt (data build tool)?

dbt to framework do **transformacji danych w SQL**, który dodaje:
- **Modularność** - modele SQL referencujące inne modele
- **Testowanie** - automatyczne testy danych (not null, unique, relationships)
- **Dokumentacja** - auto-generowana dokumentacja z DAG
- **Wersjonowanie** - modele SQL w Git
- **Materializations** - tabele, widoki, incremental

### dbt + Trino = potężna kombinacja

```
┌─────────────────────────────────────────────────┐
│                dbt Project                       │
│                                                  │
│  models/                                         │
│  ├── staging/                                    │
│  │   ├── stg_ratings.sql    ← SELECT z Trino    │
│  │   └── stg_movies.sql     ← SELECT z Trino    │
│  ├── intermediate/                               │
│  │   └── int_movie_stats.sql                     │
│  └── marts/                                      │
│      ├── dim_movies.sql     ← tabela wymiarów   │
│      └── fct_ratings.sql    ← tabela faktów     │
│                                                  │
│  Trino wykonuje SQL → wyniki do Hive/HDFS       │
└─────────────────────────────────────────────────┘
```

### Przykładowy model dbt dla Trino:

```sql
-- models/staging/stg_ratings.sql
-- dbt model: materialized as table in hive.analytics

{{ config(
    materialized='table',
    schema='analytics'
) }}

SELECT
    user_id,
    movie_id,
    rating,
    rating_timestamp,
    YEAR(rating_timestamp) AS rating_year
FROM {{ source('movielens', 'ratings') }}
WHERE rating IS NOT NULL
```

```sql
-- models/marts/fct_movie_stats.sql
{{ config(materialized='table') }}

SELECT
    movie_id,
    COUNT(*) AS num_ratings,
    AVG(rating) AS avg_rating,
    approx_percentile(rating, 0.5) AS median_rating
FROM {{ ref('stg_ratings') }}
GROUP BY movie_id
```

### Komendy dbt:
```bash
dbt run --target trino          # wykonaj wszystkie modele
dbt test                        # uruchom testy
dbt docs generate && dbt docs serve  # dokumentacja
```

**Uwaga:** To jest wyjaśnienie konceptu - nie implementujemy dbt w tym notebooku.

In [ ]:
# Symulacja dbt-style transformacji w czystym Trino SQL
# Warstwa staging
cursor = conn.cursor()
cursor.execute("CREATE SCHEMA IF NOT EXISTS memory.analytics")

# stg_ratings - oczyszczone dane
cursor.execute("""
    CREATE TABLE IF NOT EXISTS memory.analytics.stg_ratings AS
    SELECT user_id,
           movie_id,
           rating,
           rating_timestamp
    FROM postgresql.movielens.ratings
    WHERE rating IS NOT NULL
""")

# stg_movies
cursor.execute("""
    CREATE TABLE IF NOT EXISTS memory.analytics.stg_movies AS
    SELECT movie_id,
           title,
           genres,
           REGEXP_EXTRACT(title, '\\((\\d{4})\\)', 1) AS year_str
    FROM postgresql.movielens.movies
""")

# Warstwa marts - movie stats
cursor.execute("""
    CREATE TABLE IF NOT EXISTS memory.analytics.fct_movie_stats AS
    SELECT movie_id,
           COUNT(*) AS num_ratings,
           ROUND(AVG(rating), 2) AS avg_rating,
           approx_percentile(rating, 0.5) AS median_rating
    FROM memory.analytics.stg_ratings
    GROUP BY movie_id
""")

print("dbt-style layers created in memory.analytics")
run_query_print("SHOW TABLES FROM memory.analytics")

In [ ]:
# Query the marts layer
run_query_print("""
    SELECT m.title,
           m.year_str,
           f.num_ratings,
           f.avg_rating,
           f.median_rating
    FROM memory.analytics.fct_movie_stats f
    JOIN memory.analytics.stg_movies m ON f.movie_id = m.movie_id
    WHERE f.num_ratings > 10000
    ORDER BY f.avg_rating DESC
    LIMIT 15
""")

## 6. Materialized Views

### Koncept

Materialized view to **widok z cached wynikami** - zapytanie jest wykonywane raz, wynik zapisany fizycznie.

```
Zwykły VIEW:           SELECT → wykonuje query za każdym razem
Materialized VIEW:     SELECT → zwraca zapisany wynik (szybko!)
                       REFRESH → przelicza query i aktualizuje wynik
```

### W Trino:
- Materialized views są obsługiwane przez **Hive connector**
- Dane są fizycznie zapisane na HDFS jako Parquet
- `REFRESH MATERIALIZED VIEW` przelicza dane
- Trino może automatycznie użyć materialized view zamiast bazowej tabeli (rewriting)

```sql
-- Tworzenie materialized view (wymaga Hive connector)
CREATE MATERIALIZED VIEW hive.analytics.mv_movie_stats AS
SELECT movie_id,
       COUNT(*) AS num_ratings,
       AVG(rating) AS avg_rating
FROM hive.movielens.ratings
GROUP BY movie_id;

-- Odświeżanie
REFRESH MATERIALIZED VIEW hive.analytics.mv_movie_stats;

-- Użycie - Trino może automatycznie skierować tu zapytania
SELECT * FROM hive.analytics.mv_movie_stats
WHERE num_ratings > 1000;
```

### Kiedy użyć materialized views?
- Często powtarzane, kosztowne zapytania (dashboardy)
- Agregaty na dużych tabelach
- Dane zmieniają się rzadko (odświeżanie co godzinę/dzień)

In [ ]:
# Symulacja materialized view z CTAS w memory
# (prawdziwe materialized views wymagają Hive connector)

# "Materialized view" - movie stats
start = time.time()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS memory.analytics.mv_genre_stats")
cursor.execute("""
    CREATE TABLE memory.analytics.mv_genre_stats AS
    WITH genre_exploded AS (
        SELECT r.movie_id, r.rating, genre
        FROM postgresql.movielens.ratings r
        JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
        CROSS JOIN UNNEST(SPLIT(m.genres, '|')) AS t(genre)
    )
    SELECT genre,
           COUNT(*) AS total_ratings,
           approx_distinct(movie_id) AS unique_movies,
           ROUND(AVG(rating), 3) AS avg_rating,
           approx_percentile(rating, 0.5) AS median_rating
    FROM genre_exploded
    WHERE genre != '(no genres listed)'
    GROUP BY genre
""")
create_time = time.time() - start
print(f"Materialized view created in {create_time:.2f}s")

# Szybkie zapytanie do materialized view
start = time.time()
df = run_query_print("""
    SELECT * FROM memory.analytics.mv_genre_stats
    ORDER BY total_ratings DESC
""")
query_time = time.time() - start
print(f"Query time: {query_time:.3f}s (vs {create_time:.2f}s for full computation)")

## 7. Partycjonowanie i bucketing w Hive Connector

### Partycjonowanie
Dane na HDFS mogą być **partycjonowane** - fizycznie podzielone na podkatalogi.

```
hdfs://namenode:9000/data/movielens/ratings/
├── year=2010/
│   ├── part-00000.parquet
│   └── part-00001.parquet
├── year=2011/
│   └── part-00000.parquet
└── year=2015/
    └── part-00000.parquet
```

Trino automatycznie stosuje **partition pruning** - czyta tylko potrzebne partycje.

### Tworzenie partycjonowanej tabeli w Hive z Trino:

```sql
CREATE TABLE hive.movielens.ratings_partitioned (
    user_id INTEGER,
    movie_id INTEGER,
    rating DOUBLE,
    rating_timestamp TIMESTAMP
) WITH (
    format = 'PARQUET',
    partitioned_by = ARRAY['rating_year'],
    bucketed_by = ARRAY['movie_id'],
    bucket_count = 16
);
```

### Bucketing
Bucketing dzieli dane na stałą liczbę plików wg hash klucza.
Optymalizuje JOIN i GROUP BY na tym kluczu - Trino wie, które pliki łączyć.

In [ ]:
# Sprawdź partycje istniejącej tabeli hive
# (jeśli tabela była zapisana z partycjonowaniem przez Spark)
try:
    run_query_print("""
        SELECT * FROM hive.movielens."ratings$partitions"
        LIMIT 20
    """)
except Exception as e:
    print(f"Partitions query: {e}")
    print("Tabela może nie być partycjonowana lub nie istnieć.")

In [ ]:
# Właściwości tabeli hive
try:
    run_query_print("""
        SHOW CREATE TABLE hive.movielens.ratings
    """)
except Exception as e:
    print(f"Show create table: {e}")

## 8. Optymalizacja zapytań: Predicate Pushdown i Partition Pruning

### Predicate Pushdown
Trino przesuwa filtry (WHERE) do źródła danych:
- **PostgreSQL connector**: filtr staje się częścią SQL wysyłanego do PostgreSQL
- **Hive connector**: filtr jest stosowany przy odczycie Parquet (row group filtering)

```
Bez pushdown:   PostgreSQL → [ALL rows] → Trino → FILTER
Z pushdown:     PostgreSQL → FILTER → [filtered rows] → Trino
```

### Partition Pruning
Trino czyta tylko partycje pasujące do filtra WHERE:
```sql
-- Czyta TYLKO partycję year=2015 (nie skanuje reszty)
SELECT * FROM hive.movielens.ratings_partitioned
WHERE year = 2015
```

In [ ]:
# Demonstracja predicate pushdown - sprawdź EXPLAIN
explain_result = run_query("""
    EXPLAIN
    SELECT user_id, movie_id, rating
    FROM postgresql.movielens.ratings
    WHERE user_id = 42 AND rating >= 4.0
""")

for _, row in explain_result.iterrows():
    print(row.iloc[0])

print("\n--- Szukaj 'constraint' lub 'pushdown' w planie ---")

In [ ]:
# Benchmark: pushdown vs no pushdown (symulacja)
# Z pushdown - filtr na user_id przesunięty do PostgreSQL
start = time.time()
run_query("""
    SELECT COUNT(*), AVG(rating)
    FROM postgresql.movielens.ratings
    WHERE user_id = 42
""")
pushdown_time = time.time() - start

# Bez efektywnego pushdown - obliczenie po stronie Trino
start = time.time()
run_query("""
    SELECT COUNT(*), AVG(rating)
    FROM (
        SELECT * FROM postgresql.movielens.ratings
    ) sub
    WHERE user_id = 42
""")
no_pushdown_time = time.time() - start

print(f"Z pushdown:    {pushdown_time:.3f}s")
print(f"Bez pushdown:  {no_pushdown_time:.3f}s")

## 9. Performance Tuning: Session Properties i Resource Groups

### Session Properties
Trino pozwala dostroić zachowanie per-query lub per-session.

### Resource Groups
Resource groups kontrolują alokację zasobów (CPU, pamięć) między zapytaniami.

```json
{
  "rootGroups": [
    {
      "name": "analytics",
      "maxQueued": 100,
      "hardConcurrencyLimit": 10,
      "softMemoryLimit": "80%"
    },
    {
      "name": "etl",
      "maxQueued": 10,
      "hardConcurrencyLimit": 3,
      "softMemoryLimit": "20%"
    }
  ]
}
```

In [ ]:
# Dostępne session properties
run_query_print("""
    SHOW SESSION
""")

In [ ]:
# Ustawienie session properties
cursor = conn.cursor()

# Join distribution type: AUTOMATIC, BROADCAST, PARTITIONED
cursor.execute("SET SESSION join_distribution_type = 'AUTOMATIC'")

# Join reordering: AUTOMATIC, ELIMINATE_CROSS_JOINS, NONE
cursor.execute("SET SESSION join_reordering_strategy = 'AUTOMATIC'")

# Optymalizacja hash aggregation
cursor.execute("SET SESSION task_concurrency = 8")

print("Session properties set.")

In [ ]:
# Wpływ join_distribution_type na plan zapytania
for dist_type in ['BROADCAST', 'PARTITIONED']:
    cursor = conn.cursor()
    cursor.execute(f"SET SESSION join_distribution_type = '{dist_type}'")
    
    explain = run_query(f"""
        EXPLAIN (TYPE DISTRIBUTED)
        SELECT m.title, COUNT(*) AS cnt
        FROM postgresql.movielens.ratings r
        JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
        GROUP BY m.title
        ORDER BY cnt DESC
        LIMIT 10
    """)
    
    print(f"\n=== join_distribution_type = {dist_type} ===")
    for _, row in explain.iterrows():
        print(row.iloc[0])

# Reset to AUTOMATIC
cursor.execute("SET SESSION join_distribution_type = 'AUTOMATIC'")

### Najważniejsze session properties:

| Property | Wartości | Opis |
|----------|---------|------|
| `join_distribution_type` | AUTOMATIC, BROADCAST, PARTITIONED | Strategia join |
| `join_reordering_strategy` | AUTOMATIC, NONE | Czy optimizer zmienia kolejność joinów |
| `task_concurrency` | liczba | Paralelizm per task |
| `query_max_memory` | np. '4GB' | Limit pamięci per query |
| `query_max_execution_time` | np. '10m' | Timeout query |
| `optimize_hash_generation` | true/false | Optymalizacja hash joinów |

## Zadanie 1

Przepisz zapytania z **notebook 05 (Spark SQL)** na Trino SQL.

Przepisz następujące zapytania:

1. **ROW_NUMBER** - top 3 najwyżej ocenianych filmów per użytkownik (user_id <= 10)
2. **LAG** - zmiana oceny w czasie dla user_id = 42
3. **CTE z window function** - profil użytkownika z kategorią aktywności i ranking w kategorii
4. **NTILE** - podział użytkowników na decyle po średniej ocenie, z approx_distinct zamiast COUNT(DISTINCT)

Porównaj składnię - co jest takie samo, co inne?

In [ ]:
# Twoje rozwiązanie:

# 1. ROW_NUMBER - top 3 per user
run_query_print("""

""")

In [ ]:
# 2. LAG - zmiana oceny w czasie
run_query_print("""

""")

In [ ]:
# 3. CTE z window function
run_query_print("""

""")

In [ ]:
# 4. NTILE z approx_distinct
run_query_print("""

""")

## Zadanie końcowe

Zbuduj **raport analityczny** w Trino łączący zaawansowane techniki:

### Część 1: Top filmy per gatunek per dekada
- Użyj `UNNEST(SPLIT(genres, '|'))` do rozbicia gatunków
- Wyciągnij rok z tytułu: `REGEXP_EXTRACT(title, '\(([0-9]{4})\)', 1)`
- Oblicz dekadę: `FLOOR(CAST(year AS INTEGER) / 10) * 10`
- Użyj `ROW_NUMBER()` do znalezienia top 3 filmów per gatunek per dekada
- Filtruj: minimum 100 ocen per film

### Część 2: Segmentacja użytkowników
- Podziel użytkowników na segmenty po aktywności (power/active/casual/rare)
- Dla każdego segmentu oblicz z `approx_percentile`: medianę, p25, p75 średniej oceny
- Dodaj `approx_distinct` gatunków ocenianych per segment

### Część 3: Trending analysis
- Użyj `LAG()` do porównania średniej oceny filmu z poprzednią dekadą
- Znajdź gatunki, których popularność (liczba ocen) rośnie najszybciej
- Użyj `PERCENT_RANK()` do rankingu trendów

In [ ]:
# Twoje rozwiązanie:

# Część 1: Top filmy per gatunek per dekada
run_query_print("""

""")

In [ ]:
# Część 2: Segmentacja użytkowników
run_query_print("""

""")

In [ ]:
# Część 3: Trending analysis
run_query_print("""

""")

In [ ]:
# Cleanup
cursor = conn.cursor()
for table in ['stg_ratings', 'stg_movies', 'fct_movie_stats', 'mv_genre_stats']:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS memory.analytics.{table}")
    except Exception as e:
        print(f"Could not drop {table}: {e}")

conn.close()
print("Done!")